## Programowanie dynamiczne – Wyznaczanie optymalnej wielkości partii produkcyjnej
### Szymon Wysogląd

In [1]:
import numpy as np
import pandas as pd

### Zadanie 1

In [2]:
class Magazine:
  def __init__(self, minY, maxY, y0, yEnd, n, g, h, q):
    self.minY = minY
    self.maxY = maxY
    self.y0 = y0
    self.yEnd = yEnd
    self.n = n
    self.g = g # koszty produkcji
    self.h = h # koszty magazynowania
    self.q = q # liczba potrzebnych sztuk do oddania
    self.ammounts = np.ones((maxY - minY + 1, n + 1)) * np.inf
    self.costs = np.ones((maxY - minY + 1, n + 1)) * np.inf
    self.maxProd = max(g)
    self.showInput()
  
  def __findMin(self, i, x):
    y = i + self.minY
    minF = np.inf
    minAmmount = np.inf
    sold = self.q[x]
    # jeżeli w maagzynie potrzebujemy więcej zwróc inf, czyli, że niemożliwe
    if x != self.n and y < self.q[x+1]:
      return np.inf, np.inf
    # jeżeli jest to ostatnia kolejka, to interesuje nas tylko yEnd, żeby zostało
    if x == self.n and y != self.yEnd:
      return np.inf, np.inf
    for j in range(y+1):
      # potrzebujemy y produktów, z czego j produkujemy w tym miesiącu,
      # sold wyprzedaliśmy, więc w tamtym miesiącu w magazynie musiało być y - j + sold
      ySearch = y - j + sold
      # jeżeli potrzebowalibyśmy zbyt mało lub zbyt wiele z poprzedniego miesiąca
      if ySearch < self.minY or ySearch > self.maxY or j > self.maxProd:
        continue
      index = ySearch - self.minY
      # f to: wczesniejszy koszt,  h - przechowywanie, g - cena produkcji 
      f = self.costs[index, x - 1] + self.h[y] + self.g[j]
      if f < minF:
        minF = f
        minAmmount = j
    return minF, minAmmount
  
  def dp(self):
    # uzupełnienie pierwszej kolumny
    Y,X = self.costs.shape
    # pierwsza kolumna określa warunki początkowe
    self.costs[self.y0 - self.minY, 0] = 0
    self.ammounts[self.y0 - self.minY, 0] = self.y0
    
    # pierwszy miesiąc
    lastMonth = self.y0 - self.q[1] 
    for i in range(Y):
      y = i + self.minY
      self.ammounts[i, 1] = y - lastMonth
      self.costs[i, 1] = self.h[y] + self.g[y - lastMonth]
    # zaczynamy od drugiego miesiąca
    for x in range(2, X):
      for i in range(Y):
        minF, minAmmount = self.__findMin(i, x)
        self.costs[i, x] = minF
        self.ammounts[i, x] = minAmmount
    
  def getSollution(self):
    Y, X = self.costs.shape 
    index = self.yEnd - self.minY
    cost = self.costs[index, -1]
    sollution = {}
    # przechodzimy od końca
    for x in range(0,X)[::-1]:
      sollution[x] = self.ammounts[index, x]  
      if x > 0:
        index =int( index + self.q[x] - self.ammounts[index, x])
    return sollution, cost
  
  def solve(self):
    self.dp()
    sollution, cost = self.getSollution()
    
    # Wizualizacja danych
    print(f'Macierz kosztów:')
    rowHeaders = [x for x in range(self.minY, self.maxY + 1)]
    colHeaders = [f'x{i}' for i in range(0, self.n + 1)]
    df = pd.DataFrame(self.costs, columns=colHeaders, index=rowHeaders)
    print(df,'\n')
    print(f'Optymalna macierz produkcji w danym miesiącu:')
    df = pd.DataFrame(self.ammounts, columns=colHeaders, index=rowHeaders)
    print(df,'\n')
    print(f'Liczba potrzebnych sztuk do dostarczenia w każdym miesiącu:')
    print(sollution)
    print(f'Koszt całkowity: {cost} zł')
    
    
  def showInput(self):
    print(f'Stan początkowy: {self.y0}')
    print(f'Stan końcowy: {self.yEnd}')
    print(f'Liczba miesięcy: {self.n}')
    print(f'Koszty produkcji:')
    rowHeaders = [x for x in range(0, self.maxProd + 1)]
    colHeaders = ['g(xi)[zł]']
    data = [x for _, x in self.g.items()]
    df = pd.DataFrame(np.array(data).T, columns=colHeaders, index=rowHeaders)
    print(df,'\n')
    print(f'Koszty magazynowania:')
    rowHeaders = [x for x in range(self.minY, self.maxY + 1)]
    colHeaders = ['h(yi)[zł]']
    data = [x for _, x in self.h.items()]
    df = pd.DataFrame(np.array(data).T, columns=colHeaders, index=rowHeaders)
    print(df,'\n')
    print(f'Liczba potrzebnych sztuk do dostarczenia:')
    rowHeaders = [x for x in range(1, self.n + 1)]
    colHeaders = ['q(i)[szt]']
    data = [x for _, x in self.q.items()]
    df = pd.DataFrame(np.array(data).T, columns=colHeaders, index=rowHeaders)
    print(df,'\n')

In [3]:
def test1():
  # koszt produkcji
  g = {0 : 2, 1 : 8, 2 : 12, 3 : 15, 4 : 17, 5 : 20}
  # koszt składowania
  h = {3 : 1, 4 : 1, 5 : 2, 6 : 4}
  # liczba sztuk do wyprodukowania
  q = {1 : 4, 2 : 2, 3 : 6, 4 : 5}
  minY = 3
  maxY = 6
  y0 = 6
  yEnd = 3
  n = 4
  m = Magazine(minY,maxY, y0, yEnd, n, g, h, q)
  m.dp()

#### Definicja zadania dla 6-miesięcznego okresu planowania

In [4]:
g = {0 : 2, 1 : 8, 2 : 12, 3 : 15, 4 : 17, 5 : 20, 6:22, 7:24, 8:25}
# koszt składowania
h = {3 : 1, 4 : 1, 5 : 2, 6 : 4, 7:5, 8:5}
# liczba sztuk do wyprodukowania
q = {1 : 4, 2 : 2, 3 : 6, 4 : 5, 5: 3, 6: 4}
minY = 3
maxY = 8
y0 = 7
yEnd = 6
n = 6
m = Magazine(minY,maxY, y0, yEnd, n, g, h, q)

Stan początkowy: 7
Stan końcowy: 6
Liczba miesięcy: 6
Koszty produkcji:
   g(xi)[zł]
0          2
1          8
2         12
3         15
4         17
5         20
6         22
7         24
8         25 

Koszty magazynowania:
   h(yi)[zł]
3          1
4          1
5          2
6          4
7          5
8          5 

Liczba potrzebnych sztuk do dostarczenia:
   q(i)[szt]
1          4
2          2
3          6
4          5
5          3
6          4 



### Zadanie 2

In [5]:
m.solve()

Macierz kosztów:
    x0    x1    x2    x3    x4    x5     x6
3  inf   3.0   inf   inf  60.0   inf    inf
4  inf   9.0   inf   inf  66.0  78.0    inf
5  inf  14.0   inf  49.0  71.0  82.0    inf
6  inf  19.0  27.0  53.0  75.0  86.0  104.0
7  0.0  22.0  30.0  56.0  78.0  89.0    inf
8  inf  25.0  32.0  57.0  79.0  90.0    inf 

Optymalna macierz produkcji w danym miesiącu:
    x0   x1   x2   x3   x4   x5   x6
3  inf  0.0  inf  inf  0.0  inf  inf
4  inf  1.0  inf  inf  1.0  4.0  inf
5  inf  2.0  inf  3.0  2.0  5.0  inf
6  inf  3.0  5.0  4.0  6.0  6.0  6.0
7  7.0  4.0  6.0  7.0  7.0  7.0  inf
8  inf  5.0  7.0  8.0  8.0  8.0  inf 

Liczba potrzebnych sztuk do dostarczenia w każdym miesiącu:
{6: 6.0, 5: 4.0, 4: 0.0, 3: 8.0, 2: 5.0, 1: 0.0, 0: 7.0}
Koszt całkowity: 104.0 zł


### Zadanie 3

#### Modyfikacje Zagadnienia
* Dynamiczne zapotrzebowanie i zmienne ceny: Wprowadzenie dynamicznych cen produkcji oraz kosztów magazynowania w zależności od pory roku lub sytuacji rynkowej. Zapotrzebowanie na produkty może także zmieniać się w zależności od miesiąca, co wprowadza dodatkową warstwę realności.

* Przerwy produkcyjne i awarie: Model może uwzględniać nieplanowane przerwy w produkcji (np. awarie maszyn), co wpływa na możliwości produkcyjne w danym okresie.

* Minimalne i maksymalne wielkości produkcji: Wprowadzenie ograniczeń dotyczących minimalnej i maksymalnej produkcji, które mogą być wymagane ze względów technologicznych lub rynkowych.

* Wieloetapowy proces produkcyjny: Rozszerzenie modelu o możliwość obliczeń dla wieloetapowych procesów produkcyjnych, gdzie produkty mogą przechodzić przez różne fazy produkcji z różnymi kosztami i czasami wykonania.

* Różnorodność produktów: Modelowanie sytuacji, w której firma produkuje więcej niż jeden rodzaj produktu, z możliwością przechowywania różnych produktów, które mogą mieć różne wymagania dotyczące magazynowania i sprzedaży.

* Optymalizacja transportu: Uwzględnienie kosztów transportu między różnymi lokalizacjami produkcji a magazynami, a także kosztów dystrybucji do klientów.

* Zmienne warunki początkowe i końcowe: Możliwość modyfikacji stanów początkowych i końcowych w magazynie w odpowiedzi na zmieniające się warunki rynkowe.



#### Złożoność oblczeniowa
Dla każdego miesiąca badamy wszystkie możliwe zapełnienia magazynu, gdzie aby uzyskać maksymalne zapełnienie magazynu, musimy przejrzeć wszystkie możliwe zapełnienia magazynu dla poprzedniego miesiąca. Złożoność obliczeniowa wynosi $$ O(n \cdot (\Delta y)^2)$$